<a href="https://colab.research.google.com/github/ahinagangopadhyay/Natural-Language-Processing/blob/main/Finetuning_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load the pre-trained tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Load the pre-trained model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
texts = [
    "I loved the movie. It was great!", "The food was terrible.", "The weather is okay.",
    "Absolutely fantastic performance!", "Worst customer service I’ve ever experienced.", "It was just fine, nothing special.",
    "Brilliant plot and acting!", "Not worth the time or money.", "Meh, it was average.",
    "Exceeded all my expectations.", "Completely ruined my day.", "It was neither good nor bad.",
    "An incredible experience!", "Disgusting and poorly made.", "It’s decent, I guess.",
    "Such a heartwarming story.", "Boring and predictable.", "Nothing too exciting, but not bad.",
    "A masterpiece!", "The worst product I’ve bought.", "I’m indifferent about it.",
    "Very impressive results.", "Awful and disappointing.", "It’s okay, nothing much."
]

sentiments = [
    "positive", "negative", "neutral",
    "positive", "negative", "neutral",
    "positive", "negative", "neutral",
    "positive", "negative", "neutral",
    "positive", "negative", "neutral",
    "positive", "negative", "neutral",
    "positive", "negative", "neutral",
    "positive", "negative", "neutral"
]




In [27]:
# Tokenize the text samples
encoded_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

# Extract the input IDs and attention masks
input_ids = encoded_texts['input_ids']
attention_mask = encoded_texts['attention_mask']

label_map = {"negative": 0, "neutral": 1, "positive": 2}
sentiment_labels = [label_map[s] for s in sentiments]


In [28]:
import torch.nn as nn

# Add a custom classification head on top of the pre-trained model
num_classes = len(set(sentiment_labels))
classification_head = nn.Linear(model.config.hidden_size, num_classes)

# Replace the pre-trained model's classification head with our custom head
model.classifier = classification_head


In [29]:
from transformers import DistilBertForSequenceClassification

# Fix: Load with num_labels=3
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased', num_labels=3
)

import torch
import torch.optim as optim

labels = torch.tensor(sentiment_labels, dtype=torch.long)
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

num_epochs = 41
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/41, Loss: 1.1045182943344116
Epoch 2/41, Loss: 1.0765279531478882
Epoch 3/41, Loss: 1.0354408025741577
Epoch 4/41, Loss: 1.0339151620864868
Epoch 5/41, Loss: 1.0097652673721313
Epoch 6/41, Loss: 0.9602931141853333
Epoch 7/41, Loss: 0.9495604634284973
Epoch 8/41, Loss: 0.8742498755455017
Epoch 9/41, Loss: 0.8632777333259583
Epoch 10/41, Loss: 0.8217425346374512
Epoch 11/41, Loss: 0.7712516188621521
Epoch 12/41, Loss: 0.7203304171562195
Epoch 13/41, Loss: 0.6745913624763489
Epoch 14/41, Loss: 0.6286987066268921
Epoch 15/41, Loss: 0.5561128854751587
Epoch 16/41, Loss: 0.5417612195014954
Epoch 17/41, Loss: 0.49548959732055664
Epoch 18/41, Loss: 0.456491082906723
Epoch 19/41, Loss: 0.41817060112953186
Epoch 20/41, Loss: 0.37993112206459045
Epoch 21/41, Loss: 0.3512045443058014
Epoch 22/41, Loss: 0.32517218589782715
Epoch 23/41, Loss: 0.2982794940471649
Epoch 24/41, Loss: 0.2702989876270294
Epoch 25/41, Loss: 0.2477107048034668
Epoch 26/41, Loss: 0.2248312383890152
Epoch 27/41, Loss:

In [30]:
import torch
from transformers import DistilBertTokenizer

# Define label map
label_map = {
    0: "Negative",
    1: "Neutral",
    2: "Positive"
}

# Example test texts
test_texts = ["Exceeded all my expectations.", "Completely ruined my day.", "It was neither good nor bad."]

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize and encode
encoded = tokenizer(test_texts, padding=True, truncation=True, return_tensors='pt')

# Put model in eval mode
model.eval()

# Run inference without gradient tracking
with torch.no_grad():
    outputs = model(input_ids=encoded['input_ids'], attention_mask=encoded['attention_mask'])
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

# Print results
for text, pred in zip(test_texts, predictions):
    class_index = pred.item()
    class_label = label_map[class_index]
    print(f"Input: \"{text}\" → Predicted Class: {class_index} ({class_label})")


Input: "Exceeded all my expectations." → Predicted Class: 2 (Positive)
Input: "Completely ruined my day." → Predicted Class: 0 (Negative)
Input: "It was neither good nor bad." → Predicted Class: 1 (Neutral)


In [8]:
sentiment_labels

[0, 1, 2]